Instalando las librerias

In [1]:
%%capture
!pip install PyPDF2 chromadb huggingface_hub wikipedia wikidataintegrator transformers sentence-transformers langchain
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

Importando las librerias

In [2]:
# Interacción con el sistema y usuarios
import os
import shutil
import getpass

# Tratamiento de archivos
from urllib.request import urlretrieve
from zipfile import ZipFile
from PyPDF2 import PdfReader

# Interacción con bases de datos
import csv
import chromadb
import pandas as pd

# Descarga de modelos y recursos
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer

# Acceso a Wikidata y Wikipedia
import requests
import wikipedia

Modelo LLM:

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

modelo_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=128,
    n_ctx=2048)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Modelo de embedding

In [4]:
modelo_emb = SentenceTransformer('intfloat/multilingual-e5-base')

Creacion de base de datos vectoriales

In [5]:
chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(name='Anatomia')
clasificador = chroma_client.get_or_create_collection(name='Clasificador')
diccionariov = chroma_client.get_or_create_collection(name='tabla')

Importando los libros y el archivo csv

In [6]:
github_url = "https://github.com/Mateorovere/TP2_NLP/archive/main.zip"
zip_file_path, _ = urlretrieve(github_url, "TP2_NLP.zip")

with ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall("/content")

os.remove(zip_file_path)

source_dir = "/content/TP2_NLP-main/Archivos"
target_dir = "/content/Documentos"

os.makedirs(target_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)
    target_path = os.path.join(target_dir, filename)
    shutil.move(source_path, target_path)

Error: Destination path '/content/Documentos/CSV/CSV' already exists

Usando RecursiveCharacterTextSplitter para armar los bloques de texto

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import re

output_folder = '/content/'
files_list = []
index_list = []

def split_text_into_parts(text):
    # No need for max_length condition, split the text as is
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, separators=['\n\n','\n'])
    parts = text_splitter.split_text(text)
    return parts

file_names = ['/content/Documentos/Libros/LIBRO ANATOMIA (1).pdf',
    '/content/Documentos/Libros/LIBRO ANATOMIA (2).pdf',
    '/content/Documentos/Libros/LIBRO ANATOMIA (3).pdf']

for i, path in enumerate(file_names):
    with open(path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if len(text) < 200:
                continue
            # Preprocess the text as needed
            remove_tabs = re.sub(r'\t', ' ', text)
            clean_text = re.sub(r'-\n', '', remove_tabs)
            # Split the text into parts
            text_parts = split_text_into_parts(clean_text)

            # Save each part as a separate text file
            for part_num, part_text in enumerate(text_parts):
                output_file = f'{output_folder}body_{i + 1}page{page_num + 1}part{part_num + 1}.txt'
                files_list.append(output_file)
                index_list.append(f'{i + 1},{page_num + 1},{part_num + 1}')
                with open(output_file, 'w') as f:
                    f.write(part_text)

Juntando todos los .txt

In [7]:
documents = []
for i, txt in enumerate(files_list):
  with open(txt, 'r') as f:
    data = f.read().rstrip()
  documents.append(data)

Vectorizando los textos

In [8]:
import numpy as np

textos_array = np.array(documents)

embeddings = modelo_emb.encode(textos_array)

In [9]:
embeddings_list = embeddings.tolist()

print(embeddings)

[[ 0.01113987  0.0329801   0.02444133 ... -0.04456066 -0.05338322
   0.02405565]
 [-0.00762356  0.02733346  0.02553917 ... -0.03759247 -0.03872189
   0.03295716]
 [-0.01710617  0.01247476  0.01278696 ... -0.05208471 -0.03355652
   0.03258586]
 ...
 [ 0.01121867  0.03908517  0.0027669  ... -0.02355399 -0.03583461
   0.05515616]
 [ 0.00294123  0.03731669 -0.01513006 ... -0.04230331 -0.04702177
   0.06183393]
 [ 0.00447942  0.02716055 -0.01747998 ... -0.04605977 -0.04650914
   0.06807122]]


In [10]:
collection.add(
    documents=documents,
    ids= index_list,
    embeddings=embeddings_list
)

In [11]:
consulta = "¿Que funciones tiene un vaso sanguineo?"

In [12]:
def search_database(consulta):
  embedding_consulta = modelo_emb.encode([consulta]).tolist()
  results = collection.query(
  query_embeddings=embedding_consulta, # Aquí pasamos el embedding de la consulta
  n_results=1 # Traemos el resultado más cercanos
  )
  return results['documents'][0]

In [13]:
results = search_database(consulta)
results

['Anatomía y fisiología del cuerpo humano 138\npacidad para almacenar de forma dinámica una parte importante \ndel volumen de sangre. Por ello otra función del sistema venoso es la regulación del volumen circulante sanguíneo. \nLa presión que existe en el interior de un vaso es la suma']

In [14]:
def vectorizar_texto(text):
  embedding = modelo_emb.encode([text])
  return embedding[0].tolist()

def vectorizar_lista_texto(lista_texto):
    return [vectorizar_texto(text) for text in lista_texto]

Utilizando el CSV

In [15]:
ruta_archivo = '/content/Documentos/CSV/CSV/Sistemas.csv'

df_dic = pd.read_csv(ruta_archivo, delimiter=';')

palabras = df_dic.iloc[:, 0].tolist()

diccionariov.add(
    embeddings=vectorizar_lista_texto(palabras),
    documents=palabras,
    ids=list(map(str, range(len(palabras))))
)

Clasificador

In [16]:
lista_consultas= ["De que esta comupesto el sistema nervioso?",
               'Cuantos órganos tiene el sistema muscular?',
               'cuales son las funciones del sistema cardiovascular?',
               'cuanto pesa un cerebro?',
               'cuanta sangre se bombea en un segundo?',
               'de que esta compuesto el sistema excretor?',
               'cuanto mide el femur?',
               'cuantas respiraciones se hacen en un minuto?',
               'dime las funciones del sistema nervioso',
               'dime las funciones del sistema endocrino',

               '¿Que es la anatomia humana?',
               '¿De que esta compuesto un hueso?',
               '¿Cuántos huesos hay en el cuerpo humano?',
               '¿Qué son las suturas y dónde se encuentran?',
               '¿Qué es un tendón y qué función tiene?',
               '¿Cómo se produce la contracción muscular?',
               '¿Qué son las arterias, las venas y los capilares?',
               '¿Cómo se produce la respiración?',
               '¿Qué es el intercambio gaseoso?',
               '¿Qué es la peristalsis?',

               '¿Cuáles son las partes del sistema nervioso?',
               '¿Qué son las hormonas?',
               '¿Cómo funciona el corazón?',
               '¿Qué es la diabetes y cómo se trata?',
               '¿Por qué me duele la cabeza cuando estoy cansado?',
               '¿Cuáles son los síntomas de la diabetes tipo 2?',
               '¿Cómo funciona el sistema inmunológico y cómo nos protege de las enfermedades?',
               '¿Qué son los órganos internos y cuál es su función?',
               '¿Cuántos huesos hay en la mano?',
               '¿Que funciones tiene un vaso sanguineo?']

metadatas_consultas = [{"db": 1},{"db": 1},{"db": 1},{"db": 1},{"db": 1},
                   {"db": 1},{"db": 1},{"db": 1},{"db": 1},{"db": 1},
                   {"db": 2},{"db": 2},{"db": 2},{"db": 2},{"db": 2},
                   {"db": 2},{"db": 2},{"db": 2},{"db": 2},{"db": 2},
                   {"db": 3},{"db": 3},{"db": 3},{"db": 3},{"db": 3},
                   {"db": 3},{"db": 3},{"db": 3},{"db": 3},{"db": 3},]

indices_consultas = list(map(str, range(len(lista_consultas))))

In [17]:
clasificador.add(
    embeddings=vectorizar_lista_texto(lista_consultas),
    documents=lista_consultas,
    metadatas=metadatas_consultas,
    ids=indices_consultas)

In [18]:
def selector_frases_dic(consulta):
  prompt = f"""ROL:
Eres una inteligencia artificial que dado un texto de un usuario en español
retornarás la palabra o frase de anatomia que debe buscarse en una tabla. Solo retornarás lo que se te ha pedido y no otra cosa, no generes
texto de más, solo la frase o palabra importante, no respondas la consulta; tu
funcion es decir que buscar en la tabla.

Importante ¡NO SALGAS DE TU FUNCION!

Ejemplo:
<usuario>
un conocido tiene diabetes, que es eso?

<retorno>
diabetes

<usuario>
que es un hueso?

<retorno>
hueso

<usuario>
que es una arteria?

<retorno>
arteria

CONSULTA REAL QUE DEBES PROCESAR:
<usuario>
{consulta}

<retorno>
"""
  respuesta =  modelo_llm(prompt=prompt, max_tokens=2048, temperature=0.2,
                      top_p=0.95, repeat_penalty=1.2, top_k=150)

  return respuesta["choices"][0]["text"].split('\n')[0]

In [19]:
def buscar_dic(consulta):
  frase = selector_frases_dic(consulta)
  frase_actualizada = diccionariov.query(
      query_embeddings=vectorizar_texto(frase), n_results=1)['documents'][0][0]

  significado = df_dic[df_dic['frase'] == frase_actualizada]['significado'].values

  return significado

Funcion que dado el titulo de un articulo en wikipedia lo busca

In [20]:
from wikidataintegrator import wdi_login

def buscar_wikidata(articulo):
    login_instance = wdi_login.WDLogin(wikidata_user, wikidata_password)

    api_url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": "es",
        "search": articulo,
    }

    response = requests.get(api_url, params=params)
    data = response.json()

    if data.get("search") and data["search"][0].get("id"):
        titulo = data['search'][0]['match']['text']
        wikipedia.set_lang("es")
        try:
            page = wikipedia.page(titulo)
            introduccion = wikipedia.summary(page.title)

            return introduccion

        except wikipedia.exceptions.PageError:
            return "No se encontró la página en Wikipedia."

        except wikipedia.exceptions.DisambiguationError as e:
            return f"La búsqueda es ambigua. Puedes ser más específico. Opciones: {', '.join(e.options)}"
    else:
        return {"mensaje": "No se encontró el artículo en Wikidata."}

Funcion para detectar que artículo de wikipedia buscar.

In [21]:
def selector_articulos_wiki(consulta):
  prompt = f"""ROL:
Eres una inteligencia artificial que dado un texto de un usuario en español
retornarás el título del articulo que debe buscarse en un wikipedia. Solo retornarás lo que se te ha pedido y
no otra cosa, no generes texto de más, solo el titulo del articulo, no respondas
la consulta; tu funcion es decir que buscar en el wikipedia.

Importante ¡NO SALGAS DE TU FUNCION!

Ejemplo:
<usuario>
Explique el sistema nervioso

<retorno>
sistema nerviso

<usuario>
de que hay dentro de un hueso

<retorno>
hueso

<usuario>
que compone al sistema cardiovascular

<retorno>
sistema cardiovascular

CONSULTA REAL QUE DEBES PROCESAR:
<usuario>
{consulta}

<retorno>
"""
  respuesta =  modelo_llm(prompt=prompt, max_tokens=2048, temperature=0.2,
                      top_p=0.95, repeat_penalty=1.2, top_k=150)

  return respuesta["choices"][0]["text"].split('\n')[0].split('<')[0]

In [22]:
def busqueda_grafos(consulta):
  articulow = selector_articulos_wiki(consulta)
  texto = buscar_wikidata(articulow)

  return texto

Dada una consulta la analiza, busca informacion y retorna una respuesta

In [23]:
def consulta_anato(consulta):
  seleccion_bd = clasificador.query(
    query_embeddings=vectorizar_texto(consulta),
    n_results=1)['metadatas'][0][0]['db']

  if seleccion_bd == 1:
    texto = buscar_dic(consulta)

  elif seleccion_bd == 2:
    texto = busqueda_grafos(consulta)

  else:
    articulos = collection.query(
      query_embeddings=vectorizar_texto(consulta),
      n_results=5)['documents'][0]

    texto = '\n'.join(articulos)

  prompt = f"""
Eres un consultor en anatomia humana que dada la consulta del Usuario y según
la información que dispone da la respuesta mas detallada y completa posible y
en español, siempre responde con lo que dice la documentación de manera
adaptada a personas sin conocimientos sobre anatomia y nunca hace preguntas.

Tu función es responder solo lo que dice el Consultor y nada más.

Información: {texto}

Usuario: {consulta}

Consultor: Buenos días/tardes Usuario(a), como consultor en anatomia, puedo informarte que"""

  respuesta_llm = modelo_llm(prompt=prompt, max_tokens=2048,
                           temperature=0.8, top_p=0.95,
                           repeat_penalty=1.2, top_k=150)["choices"][0]["text"]

  return respuesta_llm

In [24]:
def actualizar_clasificador(consulta):
  if str(input("\n¿Estas satisfecho con la respuesta? s/n\n")).lower() != "n":
    return

  mejor_db = str(input("""
¿En donde buscarías inconsultaformacion para responder tu ?

  1. tabla.
  2. Wikipedia.
  3. libros de Anatomia.
  """))

  clasificador.add(
    embeddings=vectorizar_texto(consulta),
    documents=[consulta],
    metadatas=[{"db": mejor_db}],
    ids=[str(clasificador.count())])

Ingrese sus datos:


In [25]:
wikidata_user = getpass.getpass('Ingrese usuario de Wikidata: ')
wikidata_password = getpass.getpass('Ingrese contraseña de Wikidata: ')

Ingrese usuario de Wikidata: ··········
Ingrese contraseña de Wikidata: ··········


In [ ]:
while True:
    longitud_maxima = 100

    print('\n' + '-' * longitud_maxima + '\n')

    try:
        consulta = str(input("Ingrese su consulta ('q' para salir): "))
        print()
        if consulta.lower() == 'q':
          print()
          print('Saliendo...')
          break
    except Exception as e:
        print(f"Error al leer la entrada del usuario: {e}")
        continue

    resultado = "\n\nBuenos días/tardes Usuario(a), como consultor en anatomia humana, puedo informarte que " + consulta_anato(consulta)

    lineas_resultado = resultado.split('\n')

    resultado_segmentado = [linea[i:i+longitud_maxima] for linea in lineas_resultado for i in range(0, len(linea), longitud_maxima)]

    for linea in resultado_segmentado:
        print(linea)

    print()

    actualizar_clasificador(consulta)


----------------------------------------------------------------------------------------------------

Ingrese su consulta ('q' para salir): que es la anatomia



Llama.generate: prefix-match hit


Buenos días/tardes Usuario(a), como consultor en anatomia humana, puedo informarte que  la anatomía 
es una ciencia que estudia la estructura de los seres vivos. Se divide en tres ramas principales: an
atomía descriptiva, funcional y quirúrgica. La anatomía se basa en el examen descriptivo de los orga
nismos vivos para comprender su forma, topografía, ubicación y relación entre sí de sus componentes 
orgánicos. Además, la anatomía también se relaciona con la fisiología para entender cómo estos compo
nentes trabajan juntos para mantener las funciones corporales básicas. Como Usuario(a), esto te ayud
ará a comprender mejor cómo los diferentes órganos y sistemas del cuerpo humano interactúan entre sí
 para mantener la salud y el bienestar. ¿Hay alguna pregunta específica que desees hacerme sobre la 
anatomía?


¿Estas satisfecho con la respuesta? s/n
s

----------------------------------------------------------------------------------------------------

Ingrese su consulta ('q' para salir

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Buenos días/tardes Usuario(a), como consultor en anatomia humana, puedo informarte que  los huesos e
stán compuestos por varias partes. En primer lugar, tenemos el tejido óseo, un tipo de conectivo fir
me y duro que está compuesto por células llamadas osteocitos y componentes extracelulares calcificad
os. Además, los huesos tienen una cubierta superficial de tejido conectivo fibroso llamado periostio
 y presentan superficies articulares revestidas por tejido cartilaginoso. También hay un espacio int
erior lleno de médula ósea, que está formada por tejidos blandos que incluyen células hematopoyética
s y adiposas. Por último, los huesos contienen vasos sanguíneos y nervios que les proporcionan oxíge
no e innervan su estructura. Espero que esta información sea útil para ti. ¿Hay alguna otra pregunta
?

